In [2]:
pip install keras

     |████████████████████████████████| 1.4 MB 7.9 MB/s            
You should consider upgrading via the '/N/u/nmalpani/Carbonate/miniconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

     |████████████████████████████████| 497.6 MB 21 kB/s              
     |████████████████████████████████| 57 kB 3.2 MB/s             
     |████████████████████████████████| 4.5 MB 65.4 MB/s            
     |████████████████████████████████| 42 kB 573 kB/s             
     |████████████████████████████████| 5.8 MB 35.4 MB/s            
     |████████████████████████████████| 462 kB 66.5 MB/s            
     |████████████████████████████████| 14.5 MB 43.3 MB/s            
     |████████████████████████████████| 2.1 MB 65.2 MB/s            
     |████████████████████████████████| 65 kB 496 kB/s             
     |████████████████████████████████| 81 kB 6.4 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=bf0a219b690db249ed170e3e5b22213855ee9b23afa7bb28dd59a22647c4cac3
  Stored in directory: /geode2/home/u060/nmalpani/Carbonate/.cache/pip/wheels/b6/0d/90/0d1bbd99855f99cb2f6c2e5ff96f

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



In [30]:
df = pd.read_csv('corpus.csv')
df.dropna(inplace=True)
corpus = list(df['messages'])

In [31]:
# cv = CountVectorizer(max_features = 20000)
# X = cv.fit_transform(corpus).toarray()
# y = df['target'].values

In [32]:
X = df.messages
y = df.target

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30, stratify = y)

In [34]:
max_words = 20000
max_len = 15000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
print('sequences[2] length: ', len(sequences[2]))
print('sequences length: ', len(sequences))

sequences[2] length:  94
sequences length:  36031


In [35]:

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,64,input_length=max_len)(inputs)

    layer = LSTM(64)(layer)
    layer = Dense(64,name='FC1')(layer)
    layer = Activation('leaky_relu')(layer)
    layer = Dropout(0.4)(layer)

    layer = Dense(32,name='FC2')(layer)
    layer = Activation('leaky_relu')(layer)
    layer = Dropout(0.6)(layer)


    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [36]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 15000)]           0         
                                                                 
 embedding_3 (Embedding)     (None, 15000, 64)         1280000   
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 FC1 (Dense)                 (None, 64)                4160      
                                                                 
 activation_11 (Activation)  (None, 64)                0         
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 FC2 (Dense)                 (None, 32)                2080

In [ ]:
# model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
#           validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
model.fit(sequences_matrix,Y_train,batch_size=64,epochs=1,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

 66/451 [===>..........................] - ETA: 2:59:20 - loss: 0.5694 - accuracy: 0.6951

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

242/242 [==============================] - 6s 24ms/step - loss: 0.2928 - accuracy: 0.8239


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.293
  Accuracy: 0.824


In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
id = test['Id']
test = test[['Subject','Body']]
test['messages'] = test['Subject']+' '+test['Body']

In [ ]:
test_sequences = tok.texts_to_sequences(test['messages'].astype(str))
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
test_prob = model.predict(test_sequences_matrix)

In [ ]:
submission = pd.DataFrame()
submission['Id'] = id
submission['Flag'] = test_prob.flatten()
submission = submission.set_index('Id')


In [ ]:
submission.to_csv('submission.csv')